In [ ]:
# Demo version, e_RF_DT_org-dec.ipynb
# ( begin retroactive / template note compilation versioning if needed )

# run classical ML on both original and decoded samples
# output to d_F1_clf plot series, plot v16 for naming on the three-experiment audit

# From e_RF_original.ipynb, re-run to fix gitignore error, 2022-11-11

In [1]:
%whos

Interactive namespace is empty.


In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score

from statistics import mean
from statistics import stdev

In [3]:
plt_v = 16 # Plot set version for file naming
re_samp = 5 # Resample at 100%, to train-test splits
trn_tst_splts = 5

#### Original stacked F1 production for d_train-on-one experiment

In [25]:
org = pd.read_csv('data_5k_stacked/stacked_mad_5k.tsv', sep="\t", index_col=0)

In [ ]:
# This object is named 'stacked' in the b_step_up_02.ipynb for experiment 2 (step-up)
# Want to rename the train-on-one notebook as such, from trn_one_prdct_anoth series
# Need to split-off the train-on-all

# ^ conclude the three experiments

# Plot version set is 16 for this 2022-11-10 audit

# Try reverse versioning from demo versions
# Outline of target demo versions:
    # -
    
    
    # e_RF_original is template for e_clf_org-dec.ipynb - 

In [6]:
org

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-OR-A5J1,ACC_2,2.031600e-07,2.623902e-05,0.000150,0.000808,0.000000e+00,0.000000e+00,0.000000e+00,0.000015,0.000080,...,0.246175,0.052257,0.402793,0.076887,0.071973,0.081438,0.056327,0.093008,0.162265,0.003660
TCGA-OR-A5J2,ACC_2,7.379971e-07,2.040803e-06,0.001319,0.010417,5.500307e-07,3.562391e-07,0.000000e+00,0.002795,0.000026,...,0.057344,0.078116,0.198127,0.077655,0.297834,0.089428,0.075376,0.138143,0.160813,0.001852
TCGA-OR-A5J3,ACC_3,7.518569e-07,3.670590e-05,0.000360,0.004016,8.093709e-07,1.747427e-07,0.000000e+00,0.000126,0.000011,...,0.457210,0.047086,0.499732,0.093919,0.147129,0.201959,0.094657,0.173776,0.286364,0.001353
TCGA-OR-A5J5,ACC_3,4.412304e-05,3.773372e-05,0.000473,0.008984,0.000000e+00,2.284484e-07,0.000000e+00,0.000008,0.000016,...,0.145073,0.049122,0.383014,0.090268,0.075778,0.107083,0.091087,0.124191,0.161250,0.003433
TCGA-OR-A5J6,ACC_1,1.575903e-07,1.258983e-06,0.000116,0.000408,0.000000e+00,0.000000e+00,0.000000e+00,0.000417,0.000039,...,0.083668,0.039649,0.346296,0.090104,0.095647,0.080974,0.319514,0.106429,0.525836,0.034673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-WC-AA9E,UVM_3,1.249358e-07,2.495213e-07,0.000024,0.000209,2.088109e-05,1.463503e-05,1.682100e-05,0.002865,0.000019,...,0.047880,0.092898,0.113284,0.123114,0.155735,0.117742,0.123260,0.062998,0.276482,0.002231
TCGA-YZ-A980,UVM_1,1.158981e-07,3.588083e-06,0.000569,0.001479,8.421556e-07,1.212139e-07,3.587013e-07,0.000017,0.000107,...,0.054554,0.064715,0.230837,0.082319,0.094013,0.088344,0.063399,0.080400,0.303070,0.002967
TCGA-YZ-A982,UVM_2,0.000000e+00,2.492115e-07,0.000045,0.000326,4.835462e-06,2.087946e-06,0.000000e+00,0.000022,0.000101,...,0.067079,0.038145,0.199467,0.107931,0.094532,0.071734,0.092176,0.056409,0.306822,0.000594
TCGA-YZ-A983,UVM_3,8.682484e-08,2.202129e-05,0.000151,0.000401,3.658971e-05,7.445660e-06,3.520030e-05,0.000045,0.000028,...,0.041510,0.061564,0.189872,0.086804,0.139787,0.162714,0.070033,0.072191,0.201858,0.001488


In [7]:
ls

Casey_Greene_baseline.ipynb
Classical ML on raw vs decoded notes.rtf
Code_demo_2022-11-02.rtf
J-Lab_Exa_blog.txt
RNB00978_runtimes/
a_ntrsct_00.ipynb
a_step_up_mad5k.ipynb
archive/
b_model_trn_one_prdct_anoth_00.ipynb
b_model_trn_one_prdct_anoth_01.ipynb
b_step_up_00.ipynb
b_step_up_01.ipynb
b_step_up_02.ipynb
baseline_00.ipynb
baseline_01.ipynb
baseline_02.ipynb
baseline_03.ipynb
baseline_04.ipynb
baseline_05.ipynb
baseline_overlay/
clf_org_dcd_ovrly.ipynb
d_org-dcd.ipynb
d_original.ipynb
d_t1pa_ovrly.ipynb
data/
data_5k/
data_5k_indv/
data_5k_stacked/
data_miR/
decoded/
decoded_baseline_F1s/
decoded_baseline_plots/
e_RF_original.ipynb
hold/
mad5k.ipynb
matrices_build.ipynb
non-RNA.ipynb
normalizer.ipynb
original_baseline_F1s/
original_baseline_plots/
readme.txt
runtime_check-Copy1.ipynb
step-up_loss_plots/
t1pa_F1s/
t1pa_decoded/
t1pa_plots/
train-on-all_latent_objects/
train-on-all_loss_plots/
train-on-one_loss_plots/
train_on_all_decoded/
train_on_one_decoded/
transfer_learning_los

In [8]:
mkdir original_stacked_F1s

### Above is target read dir for all F1 classifcation plots
This is the classifier notebook on the original mad 5k stacked data

In [ ]:
# Time to deal with the primary tumor type helper column
    # Get the templates from step_up v2

# using plot version 16 for the redo on stacked 5k mad train-on-one experiment

In [9]:
org.Labels.str.split('_').str[0].unique() # to code snippet templates

array(['ACC', 'BLCA', 'BRCA', 'CESC', 'COADREAD', 'ESCC', 'GEA', 'HNSC',
       'KIRCKICH', 'KIRP', 'LIHCCHOL', 'LUAD', 'LUSC', 'MESO', 'OV',
       'PAAD', 'PCPG', 'PRAD', 'SARC', 'SKCM', 'TGCT', 'THCA', 'THYM',
       'UCEC', 'UVM'], dtype=object)

In [36]:
org.Labels.str.split('_').str[0].unique()[:2]

array(['ACC', 'BLCA'], dtype=object)

In [26]:
# 2022-11-10 audit rebuil, to demo version of classifier e_

RF_blnk_frm = pd.DataFrame()
DT_blnk_frm = pd.DataFrame()

# for c in org.Labels.str.split('_').str[0].unique()[:2]: # use c for cohort
for c in org.Labels.str.split('_').str[0].unique(): # use c for cohort
    print(c)
    primary_subset = org[org.Labels.str.contains(c)]
    
    re_samp = re_samp
    trn_tst_splts = trn_tst_splts
    
    RF_F1_dct = {} # to a row to stack into the blank frame, refresh at the start of each cancer row
    DT_F1_dct = {}
    
    break
    for j in list(range(0, re_samp)): # step through five resamplings
        X = primary_subset.iloc[:, 1:] # take all the samples in each of five times through the loop
        y = primary_subset.iloc[:, 0]
        
        RF_output = [] # tack on five tt predictions then reset for next resampling
        DT_output = []
        
        for i in list(range(0, trn_tst_splts)): # five train-test splits
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            RF_clf = RandomForestClassifier(max_depth=2) # intitialize a fresh clf each tt splt
            DT_clf = DecisionTreeClassifier()            
            
            RF_clf.fit(X_train, y_train)
            DT_clf.fit(X_train, y_train)
            
            RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
            RF_output.append(RF_F1_scr)
            
            DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
            DT_output.append(DT_F1_scr) # end tt loop
                                  
        RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
        RF_F1_dct['std'+str(j)] = stdev(RF_output)
        
        DT_F1_dct['mean'+str(j)] = mean(DT_output)
        DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack

    RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_' + str(RF_clf)]) # back to cancer loop, make a row DF
    RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] ) # Put the cancer to the blank storage frame
    
    DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_' + str(DT_clf)])
    DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )    
        
    # break # c
print('all done') # still no auto-file writing, interactive mode

ACC
all done


In [7]:
%whos

Variable                 Type         Data/Info
-----------------------------------------------
DecisionTreeClassifier   ABCMeta      <class 'sklearn.tree._cla<...>.DecisionTreeClassifier'>
RandomForestClassifier   ABCMeta      <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
f1_score                 function     <function f1_score at 0x147dfa310>
mean                     function     <function mean at 0x179bf3dc0>
pd                       module       <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
plt_v                    int          16
re_samp                  int          5
stdev                    function     <function stdev at 0x179bf65e0>
train_test_split         function     <function train_test_split at 0x147bbb430>
trn_on_one_dec           DataFrame                  Labels  N:G<...>[399 rows x 5001 columns]
trn_tst_splts            int          5


In [28]:
primary_subset.shape

(76, 5001)

In [27]:
primary_subset.tail(1) # this is the decoded F1 generator, was run in d_original, just copy down and clean up retroactively

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-PK-A5HB,ACC_2,0.000005,0.000006,0.00186,0.004433,0.0,0.0,0.0,0.000008,0.00001,...,0.046216,0.07366,0.241089,0.158638,0.064075,0.207109,0.084056,0.168372,0.242068,0.009073


In [52]:
RF_chrt_frm

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UVM_RandomForestClassifier(max_depth=2),0.912437,0.0522,0.909006,0.062281,0.83874,0.157132,0.838622,0.073572,0.932827,0.076454


In [53]:
DT_chrt_frm

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UVM_DecisionTreeClassifier(),0.727232,0.116942,0.677946,0.116461,0.765855,0.06267,0.702749,0.105404,0.739251,0.139505


In [43]:
RF_blnk_frm # looks good, pull the breaks, write to file, prep to create and read three decoded versions

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_RandomForestClassifier(max_depth=2),0.659503,0.109615,0.575275,0.142284,0.676474,0.141514,0.635788,0.072403,0.605985,0.092675
BLCA_RandomForestClassifier(max_depth=2),0.632672,0.047649,0.567222,0.047515,0.625502,0.090016,0.622351,0.055545,0.581160,0.075009


In [44]:
DT_blnk_frm

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_DecisionTreeClassifier(),0.647996,0.111164,0.567689,0.092232,0.530089,0.069986,0.639115,0.100779,0.440670,0.176188
BLCA_DecisionTreeClassifier(),0.695422,0.036382,0.670929,0.016514,0.674870,0.030757,0.679293,0.052988,0.680882,0.020073


In [54]:
RF_blnk_frm.to_csv( # fail
        'original_stacked_F1s/'+'RF'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t')

In [55]:
rf_bfr = pd.read_csv('original_stacked_F1s/'+'RF'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t', index_col = 0) # rando forst blank frame read

In [56]:
rf_bfr.head(1)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_RandomForestClassifier(max_depth=2),0.658843,0.1169,0.628328,0.06689,0.536475,0.185388,0.742396,0.063506,0.721789,0.06964


In [58]:
rf_bfr.tail(2)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UCEC_RandomForestClassifier(max_depth=2),0.552461,0.042559,0.536332,0.052664,0.570853,0.020523,0.555595,0.045932,0.511003,0.080863
UVM_RandomForestClassifier(max_depth=2),0.912437,0.052200,0.909006,0.062281,0.838740,0.157132,0.838622,0.073572,0.932827,0.076454


In [57]:
rf_bfr.shape

(25, 10)

In [49]:
rf_bfr

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_RandomForestClassifier(max_depth=2),0.659503,0.109615,0.575275,0.142284,0.676474,0.141514,0.635788,0.072403,0.605985,0.092675
BLCA_RandomForestClassifier(max_depth=2),0.632672,0.047649,0.567222,0.047515,0.625502,0.090016,0.622351,0.055545,0.581160,0.075009


In [59]:
DT_blnk_frm.to_csv( # fail
        'original_stacked_F1s/'+'DT'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t')

In [60]:
dt_bfr = pd.read_csv('original_stacked_F1s/'+'DT'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t', index_col = 0) # rando forst blank frame read

In [61]:
dt_bfr.head(2)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_DecisionTreeClassifier(),0.625967,0.082430,0.592762,0.065542,0.567580,0.045645,0.536205,0.059209,0.600053,0.070981
BLCA_DecisionTreeClassifier(),0.721054,0.039388,0.682029,0.048497,0.661031,0.012896,0.698350,0.020742,0.715822,0.014473


In [62]:
dt_bfr.tail(2)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UCEC_DecisionTreeClassifier(),0.529796,0.031473,0.517575,0.026576,0.537442,0.027094,0.578019,0.063398,0.534677,0.049953
UVM_DecisionTreeClassifier(),0.727232,0.116942,0.677946,0.116461,0.765855,0.062670,0.702749,0.105404,0.739251,0.139505


In [63]:
dt_bfr.shape

(25, 10)

In [ ]:
# End RF and DT on original

#### Train-on-one F1 production for d_train-on-one experiment

Target dir for F1s: 

In [8]:
mkdir train-on-one_F1s

In [4]:
trn_on_one_dec = pd.read_csv('train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv', sep="\t", index_col=0)

In [29]:
trn_on_one_dec

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-2F-A9KO,BLCA_2,0.266226,0.294665,0.287804,0.287290,0.299087,0.262060,0.260004,0.270740,0.284690,...,0.371561,0.321058,0.462665,0.348813,0.347686,0.342063,0.333076,0.373408,0.405937,0.276956
TCGA-2F-A9KP,BLCA_2,0.285378,0.285905,0.283878,0.297263,0.296055,0.277293,0.270595,0.276574,0.266651,...,0.406567,0.321803,0.461659,0.347546,0.349164,0.362311,0.332299,0.363754,0.402099,0.280254
TCGA-2F-A9KQ,BLCA_2,0.329344,0.357496,0.338200,0.336121,0.350700,0.336602,0.339676,0.324306,0.324085,...,0.405215,0.360598,0.476952,0.387721,0.389815,0.392136,0.395932,0.413052,0.430038,0.344641
TCGA-2F-A9KR,BLCA_1,0.271298,0.312547,0.283140,0.283972,0.306987,0.280454,0.274627,0.272107,0.279609,...,0.376178,0.331563,0.469330,0.365567,0.353976,0.358204,0.337693,0.380484,0.411087,0.272624
TCGA-2F-A9KT,BLCA_1,0.307971,0.322204,0.302893,0.322415,0.311992,0.308697,0.296239,0.307042,0.303178,...,0.416961,0.342644,0.466223,0.369281,0.372384,0.369681,0.347804,0.390589,0.421292,0.298631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA56,BLCA_1,0.292657,0.317755,0.299830,0.313446,0.314278,0.307970,0.286017,0.298050,0.300637,...,0.407025,0.341444,0.461919,0.361329,0.381027,0.387305,0.352206,0.393642,0.422552,0.299662
TCGA-ZF-AA58,BLCA_1,0.234231,0.258396,0.246750,0.256307,0.259360,0.251576,0.234416,0.238788,0.240425,...,0.363516,0.296872,0.455211,0.318696,0.337239,0.338449,0.310399,0.346221,0.390676,0.235789
TCGA-ZF-AA5H,BLCA_1,0.221841,0.250364,0.238526,0.261122,0.251615,0.247287,0.231672,0.240933,0.236034,...,0.362792,0.295693,0.463476,0.309523,0.342166,0.329351,0.299656,0.335697,0.396835,0.227859
TCGA-ZF-AA5N,BLCA_2,0.317559,0.325121,0.326246,0.334373,0.325583,0.345065,0.316175,0.319461,0.314304,...,0.428102,0.348623,0.471264,0.370316,0.379281,0.396871,0.374153,0.399158,0.434218,0.323460


In [ ]:
trn_on_one_dec

In [11]:
import glob

In [13]:
len(sorted(glob.glob('train-on-one_decoded/*.tsv')))

600

In [14]:
trn_on_one_paths = sorted(glob.glob('train-on-one_decoded/*.tsv'))

In [15]:
trn_on_one_paths

['train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_BRCA_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_CESC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_COADREAD_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_ESCC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_GEA_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_HNSC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_KIRCKICH_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_KIRP_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LIHCCHOL_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LUAD_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LUSC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_MESO_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_OV_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_PAAD

In [16]:
trn_on_one_paths[0]

'train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv'

In [19]:
path.split('/')[1].split('_')[0]

'ACC'

In [21]:
cohorts = []
for path in trn_on_one_paths:
    # print(path)
    # c=path.split
    cohorts.append(path.split('/')[1].split('_')[0])
    # break

In [23]:
cohorts = set(cohorts)

In [24]:
cohorts

{'ACC',
 'BLCA',
 'BRCA',
 'CESC',
 'COADREAD',
 'ESCC',
 'GEA',
 'HNSC',
 'KIRCKICH',
 'KIRP',
 'LIHCCHOL',
 'LUAD',
 'LUSC',
 'MESO',
 'OV',
 'PAAD',
 'PCPG',
 'PRAD',
 'SARC',
 'SKCM',
 'TGCT',
 'THCA',
 'THYM',
 'UCEC',
 'UVM'}

In [47]:
list(cohorts).sort()

In [49]:
clist = list(cohorts)

In [50]:
clist.sort()

In [51]:
clist

['ACC',
 'BLCA',
 'BRCA',
 'CESC',
 'COADREAD',
 'ESCC',
 'GEA',
 'HNSC',
 'KIRCKICH',
 'KIRP',
 'LIHCCHOL',
 'LUAD',
 'LUSC',
 'MESO',
 'OV',
 'PAAD',
 'PCPG',
 'PRAD',
 'SARC',
 'SKCM',
 'TGCT',
 'THCA',
 'THYM',
 'UCEC',
 'UVM']

In [ ]:
# referring to b_model_trn_one_prdct_anoth_01.ipynb

In [30]:
%whos

Variable                 Type         Data/Info
-----------------------------------------------
DT_F1_dct                dict         n=0
DT_blnk_frm              DataFrame    Empty DataFrame\nColumns: []\nIndex: []
DecisionTreeClassifier   ABCMeta      <class 'sklearn.tree._cla<...>.DecisionTreeClassifier'>
RF_F1_dct                dict         n=0
RF_blnk_frm              DataFrame    Empty DataFrame\nColumns: []\nIndex: []
RandomForestClassifier   ABCMeta      <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
c                        str          ACC
cohorts                  set          {'LUAD', 'TGCT', 'KIRP', <...>', 'ESCC', 'ACC', 'BRCA'}
f1_score                 function     <function f1_score at 0x147dfa310>
glob                     module       <module 'glob' from '/Use<...>9/lib/python3.9/glob.py'>
mean                     function     <function mean at 0x179bf3dc0>
org                      DataFrame                 Labels  N:GE<...>8009 rows x 5001 columns]
path      

In [31]:
plt_v

16

In [32]:
trn_tst_splts

5

In [33]:
re_samp

5

In [35]:
c

'ACC'

In [36]:
c = 'BLCA'

In [38]:
for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
    print(c_path)
    # break

train-on-one_decoded/BLCA_trained_ACC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_BRCA_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_CESC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_COADREAD_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_ESCC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_GEA_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_HNSC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_KIRCKICH_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_KIRP_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_LIHCCHOL_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_LUAD_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_LUSC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_MESO_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_OV_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_PAAD_decoded_11.32_fit_time.tsv
tr

In [ ]:
for chrt_path in [s for s in t1pa_paths if s.split('/')[1].startswith(chrt)]:

In [52]:
for c in clist:
    print(c)
    for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
        print(c_path)
    break

ACC
train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_BRCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_CESC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_COADREAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_ESCC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_GEA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_HNSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRCKICH_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRP_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LIHCCHOL_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_MESO_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_OV_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_PAAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_tr

In [ ]:
# 2022-11-11, Friday, generate F1s for input to d_train-on-one plota
RF_blnk_frm = pd.DataFrame()
DT_blnk_frm = pd.DataFrame()

# for c in org.Labels.str.split('_').str[0].unique()[:2]: # use c for cohort

for c in clist: # use c for cohort
    print(c)
    # primary_subset = org[org.Labels.str.contains(c)]
    RF_blnk_frm = pd.DataFrame()
    DT_blnk_frm = pd.DataFrame()
    # re_samp = re_samp
    # trn_tst_splts = trn_tst_splts
    for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
        print(c_path)
        c_not = c_path.split('_')[3]
        # df = pd.read_csv(c_path, sep="\t", index_col=0)
        # break
        RF_F1_dct = {} # to a row to stack into the blank frame, refresh at the start of each cancer row
        DT_F1_dct = {}

        # break
        for j in list(range(0, re_samp)): # step through five resamplings
            X = df.iloc[:, 1:] # take all the samples in each of five times through the loop
            y = df.iloc[:, 0]

            RF_output = [] # tack on five tt predictions then reset for next resampling
            DT_output = []

            for i in list(range(0, trn_tst_splts)): # five train-test splits
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y,
                        test_size=0.33,
                            )
                RF_clf = RandomForestClassifier(max_depth=2) # intitialize a fresh clf each tt splt
                DT_clf = DecisionTreeClassifier()            

                RF_clf.fit(X_train, y_train)
                DT_clf.fit(X_train, y_train)

                RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
                RF_output.append(RF_F1_scr)

                DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
                DT_output.append(DT_F1_scr) # end tt loop

            RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
            RF_F1_dct['std'+str(j)] = stdev(RF_output)

            DT_F1_dct['mean'+str(j)] = mean(DT_output)
            DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack

            
        RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'rndm_frst_clf'])
        RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )            

        
        DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'dcsn_tree_clf'])
        DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
        
        # break # c    
    # break    
    RF_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_RF_predict_on_n=24.tsv',
            sep = '\t')
    DT_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_DT_predict_on_n=24.tsv',
            sep = '\t')

print('all done') # still no auto-file writing, interactive mode

ACC
train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_BRCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_CESC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_COADREAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_ESCC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_GEA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_HNSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRCKICH_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRP_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LIHCCHOL_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_MESO_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_OV_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_PAAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_tr

In [ ]:
# Templates / notes - delete for demo version

In [ ]:
for clf_name in ['RF', 'DT']:
    blnk_frm.to_csv( # fail
        'original_stacked_F1s/'+clf_name+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t')

In [ ]:
bfr = pd.read_csv('original_stacked_F1s/5k_r13.tsv', sep = '\t', index_col = 0) # blank frame read

In [ ]:
# from b_model_trn_one_prdct_anoth_01, use to build a 5k stacked original version above:

# DELETE IN DEMO VERSION

In [ ]:
 # Original baseline F1 scores on 5k feats for plotting - v5, redoing on M1 machine
plt_v = 13         # to plot naming, 13 is first M1 version

re_samp = 5 # v13, still running at 100%
trn_tst_splts = 5 # trn_tst_splts   

blnk_frm = pd.DataFrame()

for path in paths:
    
    cohort = path.split('/')[1].split('.')[0] # should still work
    print(cohort)
    # break # check
    df = pd.read_csv(path, sep="\t", index_col=0)
    features = df.iloc[:, 1:].mad().sort_values(ascending=False)[:5000].index
    normVals = df[features].max()
    vals = df[features]
    X_5k = (vals / normVals)
    df = pd.concat( [df.Labels, X_5k] , axis = 1) # Overwrite df with 5k mad feats
    # break
    F1_dct = {}
    for j in list(range(0, re_samp)): # 100% sampling rate
        X = df.iloc[:, 1:]
        y = df.iloc[:, 0]
        output = []
        for i in list(range(0, trn_tst_splts)): # number of train test splits
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            clf = RandomForestClassifier(max_depth=2, # reset the classifier each time through loop
                )
            clf.fit(X_train, y_train)
            F1_scr = f1_score(y_test, clf.predict(X_test), average='weighted')
            output.append(F1_scr) # extracting stats and overwriting, str(clf)
                                  
        F1_dct['mean'+str(j)] = mean(output)
        F1_dct['std'+str(j)] = stdev(output)

    chrt_frm = pd.DataFrame(F1_dct, index = [cohort + '_' + str(clf)])
    blnk_frm = pd.concat( [blnk_frm, chrt_frm] )
    # print('path loop done')

print('all done') # no auto-file writing, interactive mode